In [1]:
# Auto-reload setup
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from datetime import date, datetime
sys.path.append(os.path.abspath("../src"))

from data_manipulation_func import load_epex_data, clean_data
from plotting import process_time_period
print('Successful import')
df = load_epex_data('../data/spot_prices_2025_raw.csv')
df = clean_data(df)
df.to_csv("../data/clean_data.csv", index=False)
start, end = process_time_period(('2025-01-02', '2025-03-03'))
# start = datetime.strptime('2025-01-02', "%Y-%m-%d").date()
# end = datetime.strptime('2025-03-03', "%Y-%m-%d").date()
df_period = df[(df['Date'] >= start) & (df['Date'] <= end)]
print(type(df['Date'][0]))
df_period

Successful import
<class 'datetime.date'>


,MTU (CET/CEST),Day-ahead Price (EUR/MWh),Date,Hour,Weekday,Week,Month,Weekend
24,2025-01-02 00:00:00,68.35,2025-01-02,0,Thursday,1,1,False
25,2025-01-02 01:00:00,84.57,2025-01-02,1,Thursday,1,1,False
26,2025-01-02 02:00:00,81.39,2025-01-02,2,Thursday,1,1,False
27,2025-01-02 03:00:00,50.73,2025-01-02,3,Thursday,1,1,False
28,2025-01-02 04:00:00,59.02,2025-01-02,4,Thursday,1,1,False
...,...,...,...,...,...,...,...,...
1483,2025-03-03 19:00:00,183.23,2025-03-03,19,Monday,10,3,False
1484,2025-03-03 20:00:00,159.84,2025-03-03,20,Monday,10,3,False
1485,2025-03-03 21:00:00,156.71,2025-03-03,21,Monday,10,3,False
1486,2025-03-03 22:00:00,145.09,2025-03-03,22,Monday,10,3,False
